# CAI Lab Session 1: Powerlaws

In this session you will:

- Learn about so-called power laws, a special case being Zipf's law
- Investigate whether words in text seem to fit power laws
- Preprocess text with `nltk`

## 1. On preprocessing (with `nltk`)

For this first project, you may use the very popular [natural language tooklit library](https://www.nltk.org) which provides useful language processing functionality.

In what follows you can see some examples of the type of preprocessing that we will use during the course.

- _Tokenization:_ is the process that splits a sequence of characters into a sequence of word types (or _tokens_)

- _Lower case folding_: turning all characters to lowercase

- _Stopword removal:_ removes tokens that correspond to functional words (or _stopwords_)

- _Stemming:_ mapping word types to their stem (normalization)





In [ ]:
from pprint import pprint

import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
## tokenizer

from nltk.tokenize import word_tokenize

text = """Primera parte del ingenioso hidalgo don Quijote de la Mancha.
  Capitulo primero. Que trata de la condicion y ejercicio del famoso hidalgo
  don Quijote de la Mancha"""

tokenized_text = word_tokenize(text)

pprint(tokenized_text)

In [ ]:
## stopword removal, punctuation removal, lower-case folding

from nltk.corpus import stopwords
import string

spanish_sw = set(stopwords.words('spanish') + list(string.punctuation))

print(f'there are {len(spanish_sw)} stopwords in Spanish (including punctuation), which are: {spanish_sw}')

filtered_tokenized_text = [w.lower() for w in tokenized_text if w.lower() not in spanish_sw]
pprint(filtered_tokenized_text)

In [ ]:
## stemmer

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

stemmed_text = [stemmer.stem(w) for w in filtered_tokenized_text]
pprint(stemmed_text)

In [ ]:
## useful for counting words

from collections import Counter

word_counts = Counter(stemmed_text)
pprint(word_counts)

word_counts_no_stem = Counter(filtered_tokenized_text)
pprint(word_counts_no_stem)

## 2. On powerlaws

Consider a function $y=f(x)$, that we will call a "law" relating $y$ with $x$.

__Example 1.__ You are given a bucket of some radioactive isotope. $x$ is the number of seconds since you were given
the bucket. $y$ is the number of atoms that disintegrate at the $x$th second.
Each atom decides independently from all other atoms whether to disintegrate in the next second (or
nanosecond, or whatever). From here you can see that $f$ will have the form $y=c \cdot a^{-x}$,
where $c$ depends on the number of atoms you were given, and $a$ depends on the isotope. In particular,
$a>1$ determines the half-life of the isotope.

__Example 2.__ $x$ is number of seconds and $y$ is the number of people entering a metro station during the next
10 minutes. You may have been told in statistics that $f$ is given by the Poisson distribution
(at least for ideal people and metro stations).

Many natural and artificial phenomena come in distributions that are neither exponential nor Poisson,
but so-called power laws. Intuitively, we have $y$ evolves like $x^a$, where $a$ is a constant called
the {\em exponent} of the power law. If $a$ is positive, $y$ is increasing, and if $a$ is negative,
$y$ decreases.

More precisely, a power law is
$$
y = c \cdot (x+b)^a
$$
for three constants $a$, $b$, and $c$.

There is a lot of theory about why powerlaws are ubiqutous. They are often related to self-organization,
fractality, complex dynamical systems, etc. The Web and social networks are full of powerlaws.
So is human language, social sciences (population of cities), and natural sciences (intensity of earthquakes,
relation of size and metabolism in animals).

A powerlaw with exponent $-1$ is called Zipf's law, after Mr. Zipf. By extension, sometimes powerlaws with
negative exponents are also called Zipfian laws.

In what follows we will fit some datasets to the best powerlaw that we can find - and see that they pretty close.
Note that this does NOT mean that the phenomenon generating the dataset is exactly a powerlaw.
Proving seriously that a law is (or is not) a powerlaw is another matter.


## 3. Looking at "Don Quijote"

In this exercise, we will be looking at Zipf's and Heap's text laws of the classical text _Don Quijote_. The full plain text is publicly available in many places.


In [ ]:
# load don quijote, e.g. as in:

import urllib.request

target_url = "https://fegalaz.usc.es/~gamallo/aulas/lingcomputacional/corpus/quijote-es.txt"

data = urllib.request.urlopen(target_url)

# print first N lines only as a demo
N = 10
for _ in range(N):
    line = next(data)
    print(line.decode('latin-1').strip())

---

__Exercise 1.__ Use `python` + `matplotlib` or similar library to plot the frequence of words in `don quijote` in decreasing order.

---

Is it a powerlaw? Or, can it be approximated by a powerlaw?

A trick about powerlaws is the following. Let's forget about the $b$ parameter for a second (or equivalently assume that $b = 0$),
so our powerlaw looks like
$$
y = c \cdot x^a.
$$

Taking logs on both sides, it becomes
$$
\log y = a \cdot \log x + \log c
$$

I.e., $\log y$ is a linear function of $\log x$.

In [ ]:
# getting the data & preprocessing

target_url = "https://fegalaz.usc.es/~gamallo/aulas/lingcomputacional/corpus/quijote-es.txt"
data = urllib.request.urlopen(target_url)

quijote = data.read().decode("latin-1").replace("\n", " ")
tokenized = [word for word in word_tokenize(quijote) if word not in list(string.punctuation)] #tokenize & remove punctuation
filtered_tokenized = [word.lower() for word in tokenized if word not in stopwords.words('spanish')] # remove stop words
stemmed_tokenized = [stemmer.stem(word) for word in tokenized] # stemm
stemmed_filtered_tokenized = [stemmer.stem(word) for word in filtered_tokenized]

dataset_tokenized = {
    "Tokenized": tokenized,
    "Filtered & tokenized": filtered_tokenized,
    "Stemmed & tokenized": stemmed_tokenized,
    "Stemmed, filtered & tokenized": stemmed_filtered_tokenized
}


word_frequency = {}
for name, data in dataset_tokenized.items():
    word_frequency[name] = sorted(Counter(data).items(),
                                  key=lambda item : item[1],
                                  reverse=True)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=2, subplot_titles=([name for name in word_frequency.keys()]))

for i, (name, data) in enumerate(word_frequency.items()):
    r = 1 + int(bool(max(0, i - 1)))
    c = 1 + int(bool(i % 2))
    fig.add_trace(
        go.Scatter(
            x=list(range(1, len(data) + 1)),
            y=[item[1] for item in data],
        ),
        row=r, col=c,

    )
    fig.update_xaxes(title_text="Decreasing order", row=r, col=c)
    fig.update_yaxes(title_text="Word frequency", row=r, col=c)
fig.update_layout(showlegend=False)
fig.show(renderer="png")

In [ ]:
import plotly.express as px

name = "Filtered & tokenized"
data = word_frequency[name]

fig = px.scatter(
    x=list(range(1, len(data) + 1)),
    y=[item[1] for item in data],
    labels={
        "x": "Decreasing order",
        "y": "Word frequency",
    },
    title=name
)
fig.show(renderer="png")

---

__Exercise 2.__ Now, plot the same but use _logarithmic_ $x$ and $y$ axes.

---

If our distribution is a powerlaw, this plot should be a straight line, whose slope is $a$
and intercept is $\log c$. If we put back the $b$ parameter, it distorts a bit the
low values, so in order to estimate $a$ and $c$ we have to pay attention to the large values.

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=([name for name in word_frequency.keys()]))

for i, (name, data) in enumerate(word_frequency.items()):
    r = 1 + int(bool(max(0, i - 1)))
    c = 1 + int(bool(i % 2))
    fig.add_trace(
        go.Scatter(
            x=list(range(1, len(data) + 1)),
            y=[item[1] for item in data],
            mode="markers"
        ),
        row=r, col=c
    )
    fig.update_xaxes(title_text="Decreasing order", type="log", row=r, col=c)
    fig.update_yaxes(title_text="Word frequency", type="log", row=r, col=c)
fig.update_layout(showlegend=False)
fig.show(renderer="png")

In [ ]:
import plotly.express as px

name = "Tokenized"
data = word_frequency[name]

fig = px.scatter(
    x=list(range(1, len(data) + 1)),
    y=[item[1] for item in data],
    labels={
        "x": "Decreasing order",
        "y": "Word frequency",
    },
    title=name,
    log_x=True,
    log_y=True
)
fig.show(renderer="png")

---

__Exercise 4.__ Let's find $a$ and $c$ analytically.
Assume we have $\log y = a \cdot \log x + \log c$.

Take two distinct _large_ values of $x$, find their corresponding values of $y$,
set up a system of two linear equations, and solve for $a$ and $c$.
The solution will probably not fit very well the low values of $x$. You can try to
make it better by adding the $b$ parameter, but don't agonize over it.

Alternatively, use linear regression to estimate the _slope_  ($a$ parameter) and the intercept ($\log c$ parameter).

In [ ]:
from scipy import stats
from numpy import log

x = log(list(range(1, len(data) + 1)))
y = log([item[1] for item in data])

a, logc, _, _, _ = stats.linregress(x, y)

def func(x):
  return a * x + logc

lr = list(map(func, x))

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y,
                    mode="markers",
                    name="Data"))
fig.add_trace(go.Scatter(x=x, y=lr,
                    mode="lines",
                    name="Linear regression"))

fig.update_layout(showlegend=False)
print(f"{a=}\n{logc=}")

---

__Exercise 5.__  Now, it is time to check whether Heap's law applies in the _Don Quijote_. First, plot number of different words (word _types_) as a function of text length. Next, plot it on a log-log scale. You should see a straight line. Finally, give an estimate of $\beta$ parameter of Heap's law:

$$  d = k \cdot N^{\beta} $$

---

In [ ]:
new_words = set()
new_words_counter = []

for word in tokenized:
    new_words.add(word)
    new_words_counter.append(len(new_words))

length = list(range(1, len(new_words_counter) + 1))

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Different words as a function of length", "Log-log scale"))

fig.add_trace(
    go.Scatter(
        x=length,
        y=new_words_counter,
        mode="markers"
    ),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(
        x=length,
        y=new_words_counter,
        mode="markers"
    ),
    row=1, col=2
)
fig.update_xaxes(type="log", row=1, col=2)
fig.update_yaxes(type="log", row=1, col=2)

fig.update_layout(showlegend=False)
fig.show(renderer="png")

In [ ]:
# logd = logk + beta * logN

loglength = log(length)
logcounter = log(new_words_counter)

fitloglength = list(loglength)
fitlogcounter = list(logcounter)

print(len(fitlogcounter))

i = 1000
for _ in range(376400):
    fitloglength.pop(i)
    fitlogcounter.pop(i)

print("ok")

beta, logk, _, _, _ = stats.linregress(fitloglength, fitlogcounter)

def func(x):
  return beta * x + logk

lr = list(map(func, log(length)))

fig = go.Figure()

fig.add_trace(go.Scatter(x=loglength, y=logcounter,
                    mode="markers",
                    name="Data"))
fig.add_trace(go.Scatter(x=loglength, y=lr,
                    mode="lines",
                    name="Linear regression"))

fig.show(renderer="png")

print(f"{beta=}")

## 4. Rules of delivery

- To be solved in _pairs_.

- No plagiarism; don't discuss your work with other teams. You can ask for help to others for simple things, such as recalling a python instruction or module, but nothing too specific to the session.

- If you feel you are spending much more time than the rest of the classmates, ask us for help. Questions can be asked either in person or by email, and you'll never be penalized by asking questions, no matter how stupid they look in retrospect.

- Write a short report listing the solutions to the exercises proposed. Include things like: how did you find the best $a$, $b$, $c$ parameter values for the text laws? Please include plots that compare the fitted powerlaws with the real data (preferably in log-log scale). You are welcome to add conclusions and findings that depart from what we asked you to do. We encourage you to discuss the difficulties you find; this lets us give you help and also improve the lab session for future editions.

- Turn the report to PDF. Make sure it has your names, date, and title.

- Submit your work through the [raco](http://www.fib.upc.edu/en/serveis/raco.html). There will be a `Practica` open for each report.

- Deadline: Work must be delivered __within 2 weeks__ from the end of the lab session. Late submissions risk being penalized or not accepted at all. If you anticipate problems with the deadline, tell us as soon as possible.